In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.cross_validation import random_train_test_split
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

c:\Users\502425\Anaconda3\envs\lfmrec\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
df_tracks = pd.read_csv('dfs/data_tracks.csv') #item features
df_playlists = pd.read_csv('dfs/data_playlists_tracks.csv') #user-items
df_playlists_info = pd.read_csv('dfs/data_playlists.csv') #user features
df_playlists_test = pd.read_csv('dfs/data_playlists_tracks_test.csv')
df_playlists_test_info = pd.read_csv('dfs/data_playlists_test.csv')

In [3]:
df_playlists_info.head()

#generator to get the names of the playlists in df_playlists_info
def get_playlist_names(df_playlists_info):
    for i, row in df_playlists_info.iterrows():
        yield row['name']
        

In [4]:
dataset = Dataset()
dataset.fit(df_playlists.pid, df_tracks.tid)


num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

dataset.fit_partial(users=(row['pid'] for _, row in df_playlists_info.iterrows()),
                    user_features=(row['name'] for _, row in df_playlists_info.iterrows()))

Num users: 30000, num_items 347153.


In [5]:
(interactions, weights) = dataset.build_interactions(df_playlists[['pid', 'tid']].values)
print(repr(interactions))

<30000x347153 sparse matrix of type '<class 'numpy.int32'>'
	with 2006743 stored elements in COOrdinate format>


In [6]:
df_playlists_info.head()


,pid,num_tracks,num_followers,num_edits,num_artists,num_albums,name,modified_at,duration_ms,collaborative
0,0,52,1,6,37,47,Throwbacks,1493424000,11532414,False
1,1,39,1,5,21,23,Awesome Playlist,1506556800,11656470,False
2,2,64,1,18,31,51,korean,1505692800,14039958,False
3,3,126,1,4,86,107,mat,1501027200,28926058,False
4,4,17,2,7,16,16,90s,1401667200,4335282,False


In [7]:
user_features = dataset.build_user_features([(x[0], [x[1]]) for x in df_playlists_info[['pid', 'name']].values])
print(repr(user_features))

#split the data into train and test
train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=np.random.RandomState(1337))

<30000x43602 sparse matrix of type '<class 'numpy.float32'>'
	with 60000 stored elements in Compressed Sparse Row format>


In [8]:
model = LightFM(loss='warp', no_components=100, learning_rate=0.05, random_state=1337)

model.fit(interactions, epochs=10, num_threads=8, user_features=user_features)

print("Train precision: %.2f" % precision_at_k(model, train, k=50, user_features=user_features).mean())
print("Test precision: %.2f" % precision_at_k(model, test, k=50, user_features=user_features).mean())


Train precision: 0.16
